# 使用聊天模型和提示模板构建简单的 LLM 应用程序

在本快速入门指南中，我们将展示如何使用 LangChain 构建一个简单的 LLM 应用程序。此应用程序会将英文文本翻译成另一种语言。这是一个相对简单的 LLM 应用程序——它只是一次 LLM 调用加上一些提示。尽管如此，这仍然是开始使用 LangChain 的绝佳方式——许多功能都可以通过一些提示和一次 LLM 调用来构建！

阅读本教程后，您将对以下内容有一个高层次的概述：

- 使用 [语言模型](/docs/concepts/chat_models)

- 使用 [提示模板](/docs/concepts/prompt_templates)

- 使用 [LangSmith](https://docs.smith.langchain.com/) 调试和跟踪您的应用程序

让我们开始吧！

## 设置

### Jupyter Notebook

本教程及其他教程可能在 [Jupyter notebooks](https://jupyter.org/) 中运行最为方便。在交互式环境中学习指南是更好地理解它们的好方法。有关如何安装的说明，请参见 [此处](https://jupyter.org/install)。

### 安装

要安装 LangChain，请运行：

<!-- HIDE_IN_NB
import Tabs from '@theme/Tabs';
import TabItem from '@theme/TabItem';
import CodeBlock from "@theme/CodeBlock";

<Tabs>
  <TabItem value="pip" label="Pip" default>
    <CodeBlock language="bash">pip install langchain</CodeBlock>
  </TabItem>
  <TabItem value="conda" label="Conda">
    <CodeBlock language="bash">conda install langchain -c conda-forge</CodeBlock>
  </TabItem>
</Tabs>
HIDE_IN_NB -->

In [1]:
# | output: false

# %pip install langchain
# OR
# %conda install langchain -c conda-forge

更多详情，请参阅我们的 [安装指南](/docs/how_to/installation)。

### LangSmith

你用 LangChain 构建的许多应用程序将包含多个步骤和多次对 LLM 的调用。
随着这些应用程序变得越来越复杂，能够检查你的链或代理内部具体发生了什么至关重要。
做到这一点最好的方法是使用 [LangSmith](https://smith.langchain.com)。

在上面的链接处注册后，请确保设置你的环境变量以开始记录跟踪：

```shell
export LANGSMITH_TRACING="true"
export LANGSMITH_API_KEY="..."
export LANGSMITH_PROJECT="default" # 或任何其他项目名称
```

或者，如果在笔记本中，你可以使用以下方法设置它们：

```python
import getpass
import os

try:
    # 从 .env 文件加载环境变量 (需要 `python-dotenv`)
    from dotenv import load_dotenv

    load_dotenv()
except ImportError:
    pass

os.environ["LANGSMITH_TRACING"] = "true"
if "LANGSMITH_API_KEY" not in os.environ:
    os.environ["LANGSMITH_API_KEY"] = getpass.getpass(
        prompt="请输入你的 LangSmith API 密钥 (可选): "
    )
if "LANGSMITH_PROJECT" not in os.environ:
    os.environ["LANGSMITH_PROJECT"] = getpass.getpass(
        prompt='请输入你的 LangSmith 项目名称 (默认为 "default"): '
    )
    if not os.environ.get("LANGSMITH_PROJECT"):
        os.environ["LANGSMITH_PROJECT"] = "default"
```

## 使用语言模型

首先，让我们学习如何单独使用语言模型。LangChain 支持许多不同的语言模型，您可以互换使用它们。有关开始使用特定模型的详细信息，请参阅 [支持的集成](/docs/integrations/chat/)。

<!-- HIDE_IN_NB>
import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs overrideParams={{openai: {model: "gpt-4o-mini"}}} />
HIDE_IN_NB -->

In [2]:
# | output: false
# | echo: false

from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini")

我们首先直接使用模型。`[ChatModels](/docs/concepts/chat_models)` 是 LangChain `[Runnables](/docs/concepts/runnables/)` 的实例，这意味着它们公开了一个与它们交互的标准接口。要简单地调用模型，我们可以向 `.invoke` 方法传递一个 [messages](/docs/concepts/messages/) 列表。

In [3]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage("Translate the following from English into Italian"),
    HumanMessage("hi!"),
]

model.invoke(messages)

AIMessage(content='Ciao!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 20, 'total_tokens': 23, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0705bf87c0', 'finish_reason': 'stop', 'logprobs': None}, id='run-32654a56-627c-40e1-a141-ad9350bbfd3e-0', usage_metadata={'input_tokens': 20, 'output_tokens': 3, 'total_tokens': 23, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

:::tip

如果已启用 LangSmith，我们可以看到此运行已记录到 LangSmith，并且可以看到 [LangSmith 追踪](https://smith.langchain.com/public/88baa0b2-7c1a-4d09-ba30-a47985dde2ea/r)。LangSmith 追踪报告了 [token](/docs/concepts/tokens/) 使用情况、延迟、[标准模型参数](/docs/concepts/chat_models/#standard-parameters)（如 temperature）以及其他信息。

:::

请注意，ChatModels 接收 [message](/docs/concepts/messages/) 对象作为输入，并生成 message 对象作为输出。除了文本内容，message 对象还传递对话的 [角色](/docs/concepts/messages/#role) 并包含重要数据，例如 [工具调用](/docs/concepts/tool_calling/) 和 token 使用计数。

LangChain 还通过字符串或 [OpenAI 格式](/docs/concepts/messages/#openai-format) 支持聊天模型输入。以下内容是等效的：

```python
model.invoke("Hello")

model.invoke([{"role": "user", "content": "Hello"}])

model.invoke([HumanMessage("Hello")])
```

### 串流

因为聊天模型是 [Runnables](/docs/concepts/runnables/)，所以它们公开了一个标准接口，其中包括异步和串流调用模式。这使我们能够从聊天模型串流单个 token：

In [4]:
for token in model.stream(messages):
    print(token.content, end="|")

|C|iao|!||

你可以在[此指南](/docs/how_to/chat_streaming/)中找到有关流式聊天模型输出的更多详细信息。

## 提示词模板

目前，我们将消息列表直接传递给语言模型。这个消息列表从何而来？通常，它是由用户输入和应用程序逻辑组合而成的。应用程序逻辑通常会接收原始用户输入，并将其转换为一个已准备好传递给语言模型的消息列表。常见的转换包括添加系统消息或使用用户输入格式化模板。

[提示词模板](/docs/concepts/prompt_templates/) 是 LangChain 中的一个概念，旨在协助进行此转换。它们接收原始用户输入，并返回数据（一个提示词），该数据已准备好传递给语言模型。

我们在这里创建一个提示词模板。它将接收两个用户变量：

- `language`：要将文本翻译成的语言
- `text`：要翻译的文本

In [5]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "Translate the following from English into {language}"

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

请注意，`ChatPromptTemplate` 支持在单个模板中使用多种[消息角色](/docs/concepts/messages/#role)。我们将 `language` 参数格式化为系统消息，并将用户 `text` 格式化为用户消息。

此提示模板的输入是一个字典。我们可以单独玩玩这个提示模板，看看它自己有什么作用。

In [6]:
prompt = prompt_template.invoke({"language": "Italian", "text": "hi!"})

prompt

ChatPromptValue(messages=[SystemMessage(content='Translate the following from English into Italian', additional_kwargs={}, response_metadata={}), HumanMessage(content='hi!', additional_kwargs={}, response_metadata={})])

我们可以看到它返回了一个 `ChatPromptValue`，其中包含两个消息。如果要直接访问这些消息，我们可以这样做：

In [7]:
prompt.to_messages()

[SystemMessage(content='Translate the following from English into Italian', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='hi!', additional_kwargs={}, response_metadata={})]

最后，我们可以在格式化后的提示上调用聊天模型：

In [8]:
response = model.invoke(prompt)
print(response.content)

Ciao!


:::tip
Message `content` 可以同时包含文本和具有附加结构的 [content blocks](/docs/concepts/messages/#aimessage)。更多信息请参阅[本指南](/docs/how_to/output_parser_string/)。
:::

如果我们查看 [LangSmith trace](https://smith.langchain.com/public/3ccc2d5e-2869-467b-95d6-33a577df99a2/r)，我们可以确切地看到聊天模型接收到的提示，以及 [token](/docs/concepts/tokens/) 使用信息、延迟、[standard model parameters](/docs/concepts/chat_models/#standard-parameters)（如 temperature）和其他信息。

## 结论

就是这样！在本教程中，您学习了如何创建您的第一个简单的 LLM 应用程序。您学会了如何使用语言模型，如何创建提示模板，以及如何为使用 LangSmith 构建的应用程序获得出色的可观测性。

这仅仅是您要成为一名熟练的 AI 工程师需要学习内容的冰山一角。幸运的是，我们还有许多其他资源！

如需进一步阅读 LangChain 的核心概念，请参阅我们详细的[概念指南](/docs/concepts)。

如果您对这些概念有更具体的问题，请查看以下部分的操作指南：

- [聊天模型](/docs/how_to/#chat-models)
- [提示模板](/docs/how_to/#prompt-templates)

以及 LangSmith 文档：

- [LangSmith](https://docs.smith.langchain.com)